In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/GB500/

/content/drive/MyDrive/GB500


In [ ]:
!pip install -U segmentation-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 7.6 MB/s 


In [ ]:
import glob
import random
from time import time

import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from utils import get_images, image_preprocess, dice_coef, dice_coef_loss, unir_imagem, create_folder
from unet import unet_completa
#from ActiveContourLoss import Active_Contour_Loss

In [ ]:
SEED = 42
epoch = 100

hd = np.asarray([45 , 25, 40 , 40 , 13, 40, 42 , 33, 6 , 43 , 37 , 8 , 28, 18, 46 , 37, 41 , 41 ]) #, 33, 33 ])
hu = np.asarray([112, 95, 113, 116, 91, 98, 109, 98, 67, 105, 104, 81, 97, 88, 103, 90, 110, 113]) #, 92, 101])

x_train = sorted(glob.glob('data_heart/imagesTr/*'))
y_train = sorted(glob.glob('data_heart/labelsTr/*'))
img_xtrain = get_images(x_train,hd,hu)
img_ytrain = get_images(y_train,hd,hu)

#predict_vol = []
dice_metric = []
tempo = []
local = 'outputs_unet/'

In [ ]:
from keras import backend as K
import numpy as np

def Active_Contour_Loss(y_true, y_pred, debug = False):
  """
  lenth term
  """

  #          ch, n, x, y
  #          n , x, y, ch
  x = y_pred[:,1:,:,:] - y_pred[:,:-1,:,:] # horizontal and vertical directions 
  y = y_pred[:,:,1:,:] - y_pred[:,:,:-1,:]


  delta_x = x[:,1:,:-2,:]**2
  delta_y = y[:,:-2,1:,:]**2

  delta_u = K.abs(delta_x + delta_y)

  epslon = 0.00000001
  w = 1
  lenth = w * K.mean(K.sqrt(delta_u + epslon)) # equ.(11) in the paper
  
  """
  region term
  """

  C_1 = tf.ones((256, 256))
  C_2 = tf.zeros((256, 256))
  
  region_in =   K.abs( K.mean( y_pred[:,:,:,0] * ((C_1 - y_true[:,:,:,0])**2) ) )      # equ.(12) in the paper
  region_out =  K.abs( K.mean( (1-y_pred[:,:,:,0]) * ((C_2 - y_true[:,:,:,0])**2) ) )  # equ.(12) in the paper

  lambdaP = 1 # lambda parameter could be various.
  mu = 1 # mu parameter could be various.

  eq = lenth + lambdaP * (mu * region_in + region_out)

  return eq

In [ ]:
import segmentation_models as sm
import keras
from utils import dice_coef, dice_coef_loss

keras.backend.set_image_data_format('channels_last')

#BACKBONE = ['resnet34', 'resnet50', 'resnet101', 'resnet152'][3]
#local = f'outputs_unet_{BACKBONE}/'
BACKBONE = 'resnet34'
local = f'outputs_unet_ACMLoss/'

has_backbone = True
# Active_Contour_Loss or dice_coef_loss
loss_function = Active_Contour_Loss
iteration = 0
max_iteration = 1

In [ ]:
if False:
  import tensorflow as tf
  from tensorflow.keras import backend as K
  def dice_coef(y_true, y_pred, smooth = 0.):
      var_type = tf.float64
      y_true_f = tf.cast(y_true, var_type)
      y_true_f = K.flatten(y_true_f)

      #y_pred_f = tf.cast(y_pred, var_type)
      y_pred_f = y_pred > 0.5
      #y_pred_f = tf.where(cond, y_pred, 0.0)
      #y_pred_f = tf.math.greater(y_pred, cond)

      y_pred_f = tf.cast(y_pred, var_type)
      y_pred_f = K.flatten(y_pred_f)

      intersection = K.sum(y_true_f * y_pred_f)
      return (2. * intersection + smooth) / (K.sum(y_true_f*y_true_f) + K.sum(y_pred_f*y_pred_f) + smooth)

  def dice_coef_loss(y_true, y_pred):
      return 1.-dice_coef(y_true, y_pred)

  loss_function = dice_coef_loss
  print(dice_coef(train_ground, train_ground))
  model = sm.Unet('resnet34', input_shape=(size_img, size_img, 1), classes=1, activation='sigmoid', encoder_weights=None)
  model.compile(optimizer = Adam(learning_rate = 1e-5), loss = loss_function, metrics=[dice_coef])
  history = model.fit(x = train_X, y = train_ground, batch_size = 8, epochs = epoch, callbacks = [reduce_lr, es], validation_data=(valid_X, valid_ground))

In [ ]:
#for i in range(iteration, min(iteration + 5, 10)):
for i in range(iteration, max_iteration):
    index_position = i - iteration
    random.seed(time())
    seed_min = 0
    seed_max = 2**20
    SEED = random.randint(seed_min, seed_max)
    print("Rodando Pela %i vez"%(i+1))
    train_X, valid_X, train_ground, valid_ground = train_test_split(img_xtrain, img_ytrain, test_size=0.2, random_state = SEED)
    
    size_img = 256
    
    train_X, valid_X, train_ground, valid_ground = image_preprocess(train_X, valid_X, train_ground, valid_ground, size_img = size_img)
    
    inicial = time()

    if (has_backbone):
      model = sm.Unet(BACKBONE, input_shape=(size_img, size_img, 1), classes=1, activation='sigmoid', encoder_weights=None)
      model.compile(optimizer = Adam(learning_rate = 1e-5), loss = loss_function, metrics=[dice_coef])
    else:
      model = unet_completa(size_img)
      model.compile(optimizer = Adam(learning_rate = 1e-5), loss = loss_function, metrics=[dice_coef])


    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 5, verbose = 1)
    es = EarlyStopping(monitor = 'val_loss', patience = 15, verbose = 1)

    history = model.fit(x = train_X, y = train_ground, batch_size = 8, epochs = epoch, callbacks = [reduce_lr, es], validation_data=(valid_X, valid_ground))
    final = time()
    
    np.save(f"{local}history_{i}.npy", history.history)
    model.save(local + 'Heart_Model_%i.h5'%(i))
    
    predicao = model.predict(valid_X)
    predicao = predicao > 0.5
    predicao = np.float64(predicao)

    dice_metric.append(dice_coef(predicao, valid_ground).numpy())
    print(dice_metric[index_position])
    
    img_number = 70
    #create_folder(f"{local}images")
    #unir_imagem(valid_X, valid_ground,img_number, size_img, f"{local}images/GT_img{img_number}_exec{i}")
    #unir_imagem(valid_X, predicao, img_number, size_img, f"{local}images/pred_img{img_number}_exec{i}")

    tempo_total = final - inicial
    tempo.append(tempo_total)
    print("rodou em : %f"%(tempo_total))

    with open(f'{local}Backups_{iteration}.txt', 'a') as f:
      f.write(f"Exec: {i};Dice: {dice_metric[index_position]};Tempo: {tempo_total}\n")

    K.clear_session()

np.savetxt(local + f'Dice Metric_Iter_{iteration}.txt', dice_metric)
np.savetxt(local + f'Tempo_Iter{iteration}.txt', tempo)

Rodando Pela 1 vez
Epoch 1/100
121/121 [==============================] - 37s 185ms/step - loss: 0.7292 - dice_coef: 0.2711 - val_loss: 0.7922 - val_dice_coef: 0.2058 - lr: 1.0000e-05
Epoch 2/100
121/121 [==============================] - 19s 156ms/step - loss: 0.5549 - dice_coef: 0.4455 - val_loss: 0.6562 - val_dice_coef: 0.3420 - lr: 1.0000e-05
Epoch 3/100
121/121 [==============================] - 19s 154ms/step - loss: 0.4240 - dice_coef: 0.5759 - val_loss: 0.4716 - val_dice_coef: 0.5240 - lr: 1.0000e-05
Epoch 4/100
121/121 [==============================] - 19s 153ms/step - loss: 0.3314 - dice_coef: 0.6689 - val_loss: 0.3615 - val_dice_coef: 0.6327 - lr: 1.0000e-05
Epoch 5/100
121/121 [==============================] - 18s 153ms/step - loss: 0.2745 - dice_coef: 0.7253 - val_loss: 0.2989 - val_dice_coef: 0.6960 - lr: 1.0000e-05
Epoch 6/100
121/121 [==============================] - 19s 154ms/step - loss: 0.2368 - dice_coef: 0.7632 - val_loss: 0.2608 - val_dice_coef: 0.7346 - lr: 1.